<a href="https://colab.research.google.com/github/gh-annamalai/rag-chatbot/blob/main/Langchain_With_Gemini_And_Build_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.7 MB/s eta 0:00:

In [2]:
from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
import google.generativeai as genai
from google.colab import userdata

In [4]:
import os
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

### Text Generation

In [5]:
model = genai.GenerativeModel(model_name = "gemini-pro")
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [6]:
response = model.generate_content("What are the usecases of LLMs?")

In [7]:
to_markdown(response.text)

> **Content Generation and Creation:**
> 
> * **Text writing:** Generate articles, blog posts, stories, poems, scripts, and other written content.
> * **Code generation:** Write or assist in writing code for various programming languages.
> * **Image and music generation:** Create images, artwork, and music from text or audio prompts.
> 
> **Information Retrieval and Summarization:**
> 
> * **Search engine optimization:** Improve search engine rankings by generating keyword-rich content.
> * **Question answering:** Provide answers to user queries based on large datasets of text or knowledge.
> * **Document summarization:** Summarize lengthy documents or articles into concise summaries.
> 
> **Natural Language Processing (NLP):**
> 
> * **Language translation:** Translate text between different languages.
> * **Sentiment analysis:** Determine the emotional tone or sentiment of text.
> * **Topic modeling:** Identify and extract key topics from text data.
> 
> **Customer Service and Support:**
> 
> * **Chatbots:** Engage with customers through automated conversations and provide support.
> * **Customer feedback analysis:** Analyze customer feedback to identify trends and areas for improvement.
> * **Personalized recommendations:** Provide tailored recommendations based on user preferences and past interactions.
> 
> **Research and Development:**
> 
> * **Scientific writing assistants:** Help researchers write scientific papers, proposals, and reports.
> * **Data analysis and interpretation:** Assist in analyzing large datasets and drawing meaningful conclusions.
> * **Model development:** Create new machine learning models by providing training data and defining objective functions.
> 
> **Education and Training:**
> 
> * **Personalized learning:** Adapt learning materials and assessments to individual student needs.
> * **Virtual assistants and tutors:** Provide immediate assistance and guidance to students.
> * **Language learning:** Assist in learning new languages through interactive exercises and translations.
> 
> **Finance and Business:**
> 
> * **Financial analysis:** Perform financial modeling, forecasting, and risk assessment.
> * **Market research:** Analyze consumer behavior, identify trends, and forecast market demand.
> * **Sales and negotiation:** Assist in generating leads, scheduling appointments, and negotiating deals.

### Use LangChain to Access Gemini API

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY)

In [10]:
result = llm.invoke("What are the usecases of LLMs?")


In [11]:
to_markdown(result.content)

> **Content Generation and Creation:**
> 
> * **Text generation:** Creating articles, stories, poems, marketing copy, and other written content
> * **Summarization:** Condensing text into concise summaries for quick understanding
> * **Translation:** Translating text into different languages
> * **Question answering:** Providing answers to user queries
> * **Code generation:** Generating code snippets and entire programs
> 
> **Information Retrieval and Organization:**
> 
> * **Search engine optimization (SEO):** Optimizing website content for higher search rankings
> * **Information extraction:** Extracting structured data from unstructured text, such as names, dates, and locations
> * **Data summarization:** Summarizing large datasets for easy interpretation
> * **Document classification:** Categorizing documents based on their content
> * **Knowledge management:** Organizing and retrieving knowledge from various sources
> 
> **Customer Support and Engagement:**
> 
> * **Chatbots:** Providing real-time customer support and answering user questions
> * **Sentiment analysis:** Analyzing customer feedback and identifying sentiment
> * **Personalized recommendations:** Generating personalized product or content recommendations
> * **Lead qualification:** Identifying potential leads and qualifying them for sales teams
> 
> **Education and Training:**
> 
> * **Personalized learning:** Creating tailored learning experiences based on individual student needs
> * **Assessment and grading:** Automatically grading assignments and providing feedback
> * **Educational content creation:** Developing educational materials, such as textbooks and lesson plans
> * **Virtual tutoring:** Providing virtual assistance to students in real-time
> 
> **Healthcare:**
> 
> * **Medical text generation:** Creating patient summaries, discharge instructions, and other medical documents
> * **Symptom checker:** Identifying potential diagnoses based on patient symptoms
> * **Drug discovery:** Assisting in the discovery and development of new drugs
> * **Personalized treatment plans:** Generating tailored treatment plans for individual patients
> 
> **Business and Finance:**
> 
> * **Financial analysis:** Analyzing financial data to identify trends and make predictions
> * **Market research:** Conducting market analysis and extracting insights from customer feedback
> * **Risk assessment:** Identifying and quantifying risks in business decisions
> * **Proposal generation:** Creating proposals and presentations for sales and marketing teams
> 
> **Other Use Cases:**
> 
> * **Art and music generation:** Creating artwork, music compositions, and other creative content
> * **Game development:** Designing and developing video games
> * **Social media analytics:** Analyzing social media data to gain insights into user behavior
> * **Scientific research:** Assisting in data analysis, hypothesis generation, and scientific writing

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Chat with Documents using RAG (Retreival Augment Generation)

In [14]:
#import PIL.Image

#img = PIL.Image.open('/content/rag.png')
#img

In [15]:
!sudo apt -y -qq install tesseract-ocr libtesseract-dev

!sudo apt-get -y -qq install poppler-utils libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

!pip install langchain

The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 49 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 123630 files and directories

In [17]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [20]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA



warnings.filterwarnings("ignore")
# restart python kernal if issues with langchain import.

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [21]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,
                             temperature=0.2,convert_system_message_to_human=True)


### Extract text from the PDF

In [48]:
pdf_loader = PyPDFLoader("/content/Course_4_Testing_Detection_Athlete_Rights (2).pdf")
pages = pdf_loader.load_and_split()
print(pages[1:2].page_content)


AttributeError: 'list' object has no attribute 'page_content'

In [24]:
len(pages)

3

### RAG Pipeline: Embedding + Gemini (LLM)

In [25]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [26]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)

In [27]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)

In [28]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})


In [29]:
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True

)

In [41]:
question = "What is Overview of Anti-Doping?"
result = qa_chain({"query": question})
result["result"]

'Overview of Anti-Doping is the first section of the course, which provides an introduction to the concept of anti-doping and its importance in modern sports. It covers the detection and prevention of the use of prohibited substances or methods that enhance performance in an unfair manner, including performance-enhancing drugs, blood doping, and gene doping. Thanks for asking!'

In [42]:
Markdown(result["result"])

Overview of Anti-Doping is the first section of the course, which provides an introduction to the concept of anti-doping and its importance in modern sports. It covers the detection and prevention of the use of prohibited substances or methods that enhance performance in an unfair manner, including performance-enhancing drugs, blood doping, and gene doping. Thanks for asking!

In [43]:
result["source_documents"]

[Document(metadata={}, page_content="Course 1: Introduction to Anti-Doping \n \n1.1 Overview of Anti-Doping \n Anti-doping is a critical aspect of modern \nsports, aimed at ensuring fair competition \nand protecting the health and integrity of \nathletes. \n It involves the detection and prevention of \nthe use of prohibited s ubstances or \nmethods that enhance performance in an \nunfair manner.  \n This includes performance -enhancing drugs, blood doping, and gene \ndoping. \n Anti-doping organizations, such as the World Anti -Doping Agency (WADA), \nplay a crucial role in developing and enforcing anti -doping rules and \nregulations. \n1.2 Importance of Fair Play in Sports \n Fair play is the cornerstone of \nsportsmanship and the foundation of \ncompetitive sports. \n It involves competing honestly and \nrespecting the rules and opponents. \n Doping undermines fair play by giving \nathletes an unfa ir advantage over clean \ncompetitors. \n This can erode the credibility of sports a

In [33]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [34]:
question = "Describe the Multi-head attention layer in detail?"
result = qa_chain({"query": question})
result["result"]

"I'm sorry, but the provided text does not contain any information about the Multi-head attention layer. Thanks for asking!"

In [35]:
Markdown(result["result"])

I'm sorry, but the provided text does not contain any information about the Multi-head attention layer. Thanks for asking!

In [36]:
question = "Describe Random forest?"
result = qa_chain({"query": question})
Markdown(result["result"])

I'm sorry, but the provided text does not contain any information about Random forest. Thanks for asking!